# Analisi dei Dati di Ranking e Recall

Questo notebook è dedicato all'analisi dei dati di ranking e recall relativi al progetto MIRCV. Esamina le prestazioni del sistema di ranking sviluppato, utilizzando dati estratti dai file di testo. Le seguenti funzioni personalizzate sono utilizzate per leggere, processare e preparare i dati per un'analisi dettagliata.

## Struttura del Notebook
- **Funzioni di Lettura**: Definiscono il metodo di lettura dei dati da file di testo e la loro organizzazione in strutture di dati utili.
- **Caricamento Dati**: Carica i dati di ranking e recall da file specifici per l'analisi.
- **Accesso e Analisi dei Dati**: Visualizza e analizza parti specifiche dei dati per ottenere insights iniziali sulle prestazioni del sistema di ranking.

In [ ]:
import os

In [ ]:
# Funzioni di Lettura

# read_data: Legge i dati di ranking da un file e li organizza in un dizionario
def read_data(file_path):
    """Legge i dati di ranking da un file.

    Args:
        file_path (str): Percorso del file da leggere.

    Returns:
        dict: Dizionario contenente i dati di ranking.
    """
    system_ranking = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            qid, doc_id, relevance = parts[0], str(int(parts[2])), int(parts[3])
            if qid not in system_ranking:
                system_ranking[qid] = []
            system_ranking[qid].append((doc_id, relevance))
        
    return system_ranking

# read_data_recall_base: Legge i dati di recall base da un file e li organizza in un dizionario
def read_data_recall_base(file_path):
    """Legge i dati di recall base da un file.

    Args:
        file_path (str): Percorso del file da leggere.

    Returns:
        dict: Dizionario contenente i dati di recall base.
    """
    recalls = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            qid, base = parts[0], int(parts[1])
            if qid not in recalls:
                recalls[qid] = {}
            recalls[qid] = base

    return recalls

In [ ]:
def calculate_map(system_ranking, recall_bases):
    map_scores = []

    for query in system_ranking:
        num = 0
        count = 0
        relevants = 1

        for element in system_ranking[query]:
            if count != 10:
                if element[1] > 0:
                    num += relevants / (count + 1)
                    relevants += 1
                count += 1
        
        rb = recall_bases[query]

        ap = num / rb
        map_scores.append(ap)
        print(f"AP {query}: {ap}")

    map = sum(map_scores) / len(map_scores)
    print(f"MAP: {map}")

In [ ]:
def calculate_mrr(system_ranking):
    mrr_scores = []
    for query in system_ranking:
        count = 0

        for element in system_ranking[query]:
            if element[1] > 0:
                rr = 1 / (count + 1)
                mrr_scores.append(rr)
                print(f"RR {query}: {rr}")
                break
            if count == 9:
                mrr_scores.append(0)
                print(f"RR {query}: 0")
                break
            count += 1
        
    mrr = sum(mrr_scores) / len(mrr_scores)
    print(f"MRR: {mrr}")

In [ ]:
# Caricamento Dati

# Caricamento dei dati di ranking e recall dai file specificati
system_rank_file = "./relevance_file/DAATBM25withRelevance.txt"
recall_base_file = "./qrel_file/id_counts.txt"

# Utilizzo delle funzioni di lettura per caricare i dati
system_ranking = read_data(system_rank_file)
recall_bases = read_data_recall_base(recall_base_file)

In [ ]:
# Accesso e Analisi dei Dati

# Accesso a un elemento specifico del dizionario system_ranking per l'analisi
sample_query_id = '19335'
sample_ranking_data = system_ranking[sample_query_id][0][0]

# Output: Visualizzazione dei dati di ranking per la query specifica
print(f"Dati di Ranking per la Query {sample_query_id}: {sample_ranking_data}")

In [ ]:
folder_path = "relevance_file"

for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        path = os.path.join(folder_path, filename)
        
        system_ranking = read_data(path)
        calculate_map(system_ranking, recall_bases)

In [ ]:
folder_path = "relevance_file"

for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        path = os.path.join(folder_path, filename)
        
        system_ranking = read_data(path)
        calculate_mrr(system_ranking)